In [5]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
import os  
from dotenv import load_dotenv
from src.controllers import DocumentController
from src.utils.config import get_settings

In [6]:
# Load environment variables from .env file
settings = get_settings()

# Retrieve the GROQ_API_KEY from environment variables
groq_api_key = settings.GROQ_API_KEY

In [9]:
DocumentController = DocumentController()
file_abs_path = DocumentController.get_project_path(Doc_name="health.txt")

In [12]:
loader = TextLoader(file_abs_path)
documents = loader.load()

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

In [14]:
docs = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in docs]

In [15]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Graduation project ALL 4th\Project Files\chatbot\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\Graduation project ALL 4th\Project Files\chatbot\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
db = FAISS.from_documents(
    embedding=embeddings,
    documents=docs
    )

In [17]:
query = "I have experiencing  with sleep disorders"
test_search = db.similarity_search(query)

In [18]:
search_results_content = " ".join([doc.page_content for doc in test_search])
search_results_content

"If you're experiencing sleep disorders like insomnia or sleep apnea, consult with a healthcare professional for appropriate treatment and management strategies.\n\nPreventive Care and Screening:\nRegular check-ups, screenings, and immunizations can help detect and prevent various health issues. Follow age-appropriate screening guidelines for conditions like cancer, heart disease, and diabetes. Stay up-to-date with recommended vaccinations, such as the annual flu shot and COVID-19 boosters. Sleep and Rest:\nMost adults need 7-9 hours of quality sleep per night for optimal health and functioning. Lack of sleep can lead to fatigue, impaired cognitive function, and an increased risk of various health problems. Establish a consistent sleep routine, create a sleep-conducive environment, and practice good sleep hygiene, such as avoiding caffeine and screens before bedtime. If you're experiencing persistent or severe symptoms of anxiety, depression, or other mental health issues, seek profess

In [19]:
# Define the system and human messages
system = "You are a Fitness Coach and you will answer based on hte information i give you."
human = "{text}"

# Create the chat prompt template
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# Initialize the chat model with specific parameters
chat = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

# Combine the prompt and chat into a chain
chain = prompt | chat

# Define the query for similarity search
query = "I have experiencing  with sleep disorders"

# Perform the similarity search using the database
retrived_search = db.similarity_search(query)

# Extract the page content from the search results
search_results_content = " ".join([doc.page_content for doc in retrived_search])

# Combine the search results into the prompt text
combined_text = f"{query}\n{search_results_content}"

# Invoke the chain with the combined input
response = chain.invoke({"text": combined_text})

# Print the response
print(response)


content="For someone experiencing sleep disorders like insomnia or sleep apnea, it's crucial to consult with a healthcare professional for appropriate treatment and management strategies. They may recommend lifestyle changes, medication, or therapy, such as cognitive-behavioral therapy for insomnia (CBT-I).\n\nRegarding preventive care and screening, it's essential to follow age-appropriate screening guidelines for conditions like cancer, heart disease, and diabetes. Stay updated with recommended vaccinations, such as the annual flu shot and COVID-19 boosters.\n\nSleep and rest are vital for overall health and well-being. Aim for 7-9 hours of quality sleep per night. Establish a consistent sleep routine, create a sleep-conducive environment, and practice good sleep hygiene, such as avoiding caffeine and screens before bedtime.\n\nMental health is closely linked to sleep quality. If you're experiencing persistent or severe symptoms of anxiety, depression, or other mental health issues, 

________